# **Inferring**
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Product review text

In [4]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [5]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is positive. The reviewer is satisfied with the lamp, the customer service, and the company in general.


In [6]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [7]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, pleased


## Identify anger

In [8]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [9]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [10]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


## Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.

In [11]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

Five topics discussed in the article are requested from the model in a format that each item is one or two words long and in a comma-separated list. ChatGPT returns the topics as government surveys, job satisfaction, NASA, etc.

In [12]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas without numbering them.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Government survey, Employee satisfaction, NASA, Social Security Administration, Job satisfaction


In [13]:
response.split(sep=', ')

['Government survey',
 'Employee satisfaction',
 'NASA',
 'Social Security Administration',
 'Job satisfaction']

## Make a news alert for certain topics

The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:

In [14]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [15]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as a dictionay where the key is a topic and the value is 0 or 1 for each topic if it appears.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [16]:
for i in response.split(', '):
    print(i)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [17]:
topic_dict = {
    line.split(":")[0].strip().strip('"'): int(line.split(":")[1].strip().strip(","))
    for line in response.strip().split("\n")
    if ":" in line
}

if topic_dict.get("nasa", 0) == 1:
    print("ALERT: New NASA story!")

print(topic_dict)

ALERT: New NASA story!
{'nasa': 1, 'local government': 0, 'engineering': 0, 'employee satisfaction': 1, 'federal government': 1}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

**Setup**

In [18]:
def lines_to_dict(response_text):
    topic_dict = {}
    for line in response_text.strip().split("\n"):
        if ":" in line:
            key = line.split(":")[0].strip().strip("'").strip('"')
            try:
                val = int(line.split(":")[1].strip().strip("'").strip('"'))
            except ValueError:
                val = 0
            topic_dict[key] = val
    return topic_dict

def maybe_alert(topic_dict):
    if topic_dict.get("nasa", 0) == 1:
        print("ALERT: New NASA story!")

Version 1 (Baseline)

In [19]:
headline = "NASA announces successful test of next-gen lunar lander engines."

prompt_v1 = f"""
You are an information extractor.
Given ONE news headline, output EXACTLY these four lines, nothing else:
nasa: 0 or 1
local government: 0 or 1
engineering: 0 or 1
employee satisfaction: 0 or 1

Rules:
- Output only the four lines above, in that order.
- Use 1 if the headline is substantially about the topic; otherwise 0.

Headline: "{headline}"
"""

resp_v1 = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=[{"role": "user", "content": prompt_v1}],
)

text_v1 = resp_v1.choices[0].message.content
d_v1 = lines_to_dict(text_v1)
maybe_alert(d_v1)
print(d_v1)

ALERT: New NASA story!
{'nasa': 1, 'local government': 0, 'engineering': 1, 'employee satisfaction': 0}


Version 2

In [20]:
headline = "City council approves budget for new community health centers."

prompt_v2 = f"""
You are an information extractor. Follow the format shown in the example.

Example
Headline: "SpaceX and NASA schedule crewed mission for October."
Output:
nasa: 1
local government: 0
engineering: 1
employee satisfaction: 0

Now do the same for the next headline.
Output exactly four lines and nothing else.

Headline: "{headline}"
"""

resp_v2 = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=[{"role": "user", "content": prompt_v2}],
)

text_v2 = resp_v2.choices[0].message.content
d_v2 = lines_to_dict(text_v2)
maybe_alert(d_v2)
print(d_v2)

{'nasa': 0, 'local government': 1, 'engineering': 0, 'employee satisfaction': 0}


Version 3

In [21]:
headline = "Engineers celebrate record efficiency in new solar panel design."

prompt_v3 = f"""
Classify the headline by outputting EXACTLY these four lines (no extra text):
nasa: 0 or 1
local government: 0 or 1
engineering: 0 or 1
employee satisfaction: 0 or 1

Definitions:
- nasa = space agency content (NASA missions, announcements, partnerships)
- local government = city/county councils, mayors, municipal policies
- engineering = technology, design, building, or technical methods are central
- employee satisfaction = worker morale, surveys, strikes, HR policies

Decision rule:
- Put 1 only if the topic is central to the headline (not just mentioned).
- Otherwise 0.

Headline: "{headline}"
"""

resp_v3 = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=[{"role": "user", "content": prompt_v3}],
)

text_v3 = resp_v3.choices[0].message.content
d_v3 = lines_to_dict(text_v3)
maybe_alert(d_v3)
print(d_v3)

{'nasa': 0, 'local government': 0, 'engineering': 1, 'employee satisfaction': 0}


***REPORT***

Version 1 was more of a strict format prompt
- It correctly flagged NASA
- It appears strong when given clear format instructions

Version 2 was Few-shot prompt
- It misclassified a headline as "local government"
- It shows how examples bias the model, basically it mimicked the sample more than it analyzed

Version 3 had more strict definitions
- It completely missed NASA and only picked engineering.
- Even though I gave definitions, the model still struggled, showing hallucination/misinterpretation.

What i have learned
- Prompt design matters a lot, small changes can drastically change the results
- Strict structure works best when classification is needed because heavy example prompts may confuse the model.
- Models can hallucinate or miss signals.